In [79]:
%%shell
pip install openai langchain langchain-experimental langchainhub

In [2]:
from google.colab import userdata
openai_api_key = userdata.get('openai_api_key')

In [92]:
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.tools import format_tool_to_openai_function
from langchain.agents import tool
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate
)
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents.format_scratchpad import format_to_openai_functions, format_log_to_str
from langchain.schema.agent import AgentFinish
from langchain.agents import load_tools, AgentExecutor
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description

In [5]:
llm = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)

In [7]:
@tool
def get_num_words(text: str) -> int:
    """
    This function computes the number of words in a sentence.

    Paramaters
    ----------
    text : str
        Input sentence.

    Returns
    -------
    n_words : int
        Number of words in the sentence.
    """
    return len(text.split())

In [8]:
get_num_words

StructuredTool(name='get_num_words', description='get_num_words(text: str) -> int - This function computes the number of words in a sentence.\n\n    Paramaters\n    ----------\n    text : str\n        Input sentence.\n\n    Returns\n    -------\n    n_words : int\n        Number of words in the sentence.', args_schema=<class 'pydantic.main.get_num_wordsSchemaSchema'>, func=<function get_num_words at 0x78df4c17bf40>)

In [9]:
get_num_words("hola mundo")

2

In [22]:
%%writefile main.c
#include <stdio.h>

int main(void) {
    printf("hello this is my example");
    return 0;
}

Overwriting main.c


In [23]:
%%shell
gcc main.c -o hello_world

In [24]:
!ls -l

total 24
-rwxr-xr-x 1 root root 15960 Dec 20 00:57 hello_world
-rw-r--r-- 1 root root    93 Dec 20 00:57 main.c
drwxr-xr-x 1 root root  4096 Dec 18 14:23 sample_data


In [25]:
!./hello_world

hello this is my example

In [26]:
import subprocess

In [28]:
x = subprocess.check_output("./hello_world")

In [30]:
x.decode()

'hello this is my example'

In [84]:
tools = [get_num_words]

In [85]:
render_text_description(tools)

'get_num_words: get_num_words(text: str) -> int - This function computes the number of words in a sentence.\n\n    Paramaters\n    ----------\n    text : str\n        Input sentence.\n\n    Returns\n    -------\n    n_words : int\n        Number of words in the sentence.'

In [32]:
llm_tools = llm.bind(
    functions=[format_tool_to_openai_function(tool) for tool in tools]
)

In [35]:
prompt = ChatPromptTemplate.from_messages(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You're an AI assistant, but you're not too good on word counts."
        ),
        HumanMessagePromptTemplate.from_template("{message}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

In [38]:
inputs = {
    "message": lambda x: x["message"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x["intermediate_steps"])
}

In [40]:
agent = (
    inputs |
    prompt |
    llm_tools |
    OpenAIFunctionsAgentOutputParser()
)

In [44]:
output = agent.invoke({
    "message": "Count the number of words in the sentence: 'In the river you can find water'",
    "intermediate_steps": []
})
output
#get_num_words(text="In the rver you can find water")

AgentActionMessageLog(tool='get_num_words', tool_input={'text': 'In the river you can find water'}, log="\nInvoking: `get_num_words` with `{'text': 'In the river you can find water'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "text": "In the river you can find water"\n}', 'name': 'get_num_words'}})])

In [46]:
tools_dict = {"get_num_words": get_num_words}
tool = tools_dict[output.tool]
tool

StructuredTool(name='get_num_words', description='get_num_words(text: str) -> int - This function computes the number of words in a sentence.\n\n    Paramaters\n    ----------\n    text : str\n        Input sentence.\n\n    Returns\n    -------\n    n_words : int\n        Number of words in the sentence.', args_schema=<class 'pydantic.main.get_num_wordsSchemaSchema'>, func=<function get_num_words at 0x78df4c17bf40>)

In [47]:
tool.run(output.tool_input)

7

In [49]:
# ejecuciones de herramientas anteriores (memoria)
intermediate_steps = []
# herramientas disponibles
tools_dict = {"get_num_words": get_num_words}

# invocacion inicial del modelo
output = agent.invoke({
    "message": "Count the number of words in the sentence: 'In the river you can find water'",
    "intermediate_steps": intermediate_steps
})
while not isinstance(output, AgentFinish):
    tool = tools_dict[output.tool]
    res = tool.run(output.tool_input)
    intermediate_steps.append((output, res))

    output = agent.invoke({
        "message": "Count the number of words in the sentence: 'In the river you can find water'",
        "intermediate_steps": intermediate_steps
    })

In [50]:
output

AgentFinish(return_values={'output': 'The sentence "In the river you can find water" contains 7 words.'}, log='The sentence "In the river you can find water" contains 7 words.')

In [52]:
output.return_values["output"]

'The sentence "In the river you can find water" contains 7 words.'

In [53]:
intermediate_steps

[(AgentActionMessageLog(tool='get_num_words', tool_input={'text': 'In the river you can find water'}, log="\nInvoking: `get_num_words` with `{'text': 'In the river you can find water'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "text": "In the river you can find water"\n}', 'name': 'get_num_words'}})]),
  7)]

In [60]:
get_num_words({"text": "hello world"})

2

In [62]:
llm = OpenAI(temperature=0, openai_api_key=openai_api_key)

In [86]:
tools = load_tools(tool_names=["terminal"])

In [70]:
print(tools[0].run("ls"))

hello_world
main.c
sample_data



/usr/local/lib/python3.10/dist-packages/langchain_community/tools/shell/tool.py:31: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


In [81]:
print(hub.pull("hwchase17/react").template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [82]:
template = """
Answer the following questions considering that you have access to the following tools

{tools}

Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do.
Action: the action to take, should be one of [{tool_names}].
Action Input: the input to the action.
Observation: the result of the action (this action can be repeated)
Thought: I now know the final answer.
Final Answer: the final answer of the original question.

Begin!

Question: {input}
Thought: {agent_scratchpad}
"""
prompt = PromptTemplate(
    template=template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
    )

In [73]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], template='\nAnswer the following questions considering that you have access to the following tools\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer.\nThought: you should always think about what to do.\nAction: the action to take, should be one of [{tool_names}].\nAction Input: the input to the action.\nObservation: the result of the action (this action can be repeated)\nThought: I now know the final answer.\nFinal Answer: the final answer of the original question.\n\nBegin!\n\nQuestion: {input}\nThought: {agent_scratchpad}\n')

In [83]:
render_text_description(tools)

'terminal: Run shell commands on this Linux machine.'

In [87]:
prompt_tools = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join(tool.name for tool in tools)
)

In [88]:
prompt_tools

PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': 'terminal: Run shell commands on this Linux machine.', 'tool_names': 'terminal'}, template='\nAnswer the following questions considering that you have access to the following tools\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer.\nThought: you should always think about what to do.\nAction: the action to take, should be one of [{tool_names}].\nAction Input: the input to the action.\nObservation: the result of the action (this action can be repeated)\nThought: I now know the final answer.\nFinal Answer: the final answer of the original question.\n\nBegin!\n\nQuestion: {input}\nThought: {agent_scratchpad}\n')

In [89]:
llm_stop = llm.bind(stop=["\nObservation"])

In [91]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"])
        } |
    prompt_tools |
    llm_stop |
    ReActSingleInputOutputParser()
)

In [93]:
executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

In [94]:
executor.invoke({
    "input": "List the files and folders in the current directory"
})



> Entering new AgentExecutor chain...
I need to find a way to list the files and folders in the current directory.
Action: terminal
Action Input: lshello_world
main.c
sample_data


/usr/local/lib/python3.10/dist-packages/langchain_community/tools/shell/tool.py:31: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


I now know that the current directory contains the files hello_world, main.c, and sample_data.
Final Answer: The current directory contains the files hello_world, main.c, and sample_data.

> Finished chain.


{'input': 'List the files and folders in the current directory',
 'output': 'The current directory contains the files hello_world, main.c, and sample_data.'}

In [95]:
%%shell
mkdir -p test/data/file1.txt

In [97]:
executor.invoke({
    "input": "In a sub-directory is a file named `file1.txt`, find it and give me the path of this file"
})



> Entering new AgentExecutor chain...
I need to find the file and get its path.
Action: terminal
Action Input: find . -name "file1.txt"/bin/sh: 1: Syntax error: Unterminated quoted string


/usr/local/lib/python3.10/dist-packages/langchain_community/tools/shell/tool.py:31: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


I need to use the correct syntax for the command.
Action: terminal
Action Input: find . -name 'file1.txt'./test/data/file1.txt


/usr/local/lib/python3.10/dist-packages/langchain_community/tools/shell/tool.py:31: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


I now know the path of the file.
Final Answer: ./test/data/file1.txt

> Finished chain.


{'input': 'In a sub-directory is a file named `file1.txt`, find it and give me the path of this file',
 'output': './test/data/file1.txt'}